In [1]:
import parSimClass as pc

**`parSimClass`** contains the main simulation class, **`ParticleTracing()`**, of which we will create an instance to run molecule path-tracing simulations in a given type of cell geometry. When declaring a **`ParticleTracing()`** object we wish to specify the following important parameters:

- **`flowFieldName`**: (string) the filename of the DSMC flow-field output, in the format _"directory/../DS2xyyy.DAT'_. As described in the supporting document, the file name encodes both the geometry and flow-rate of the DSMC buffer gas simulation, so the file needs to be named in this format (sorry). 


- **`NPAR`**: (integer) the number of particles to simulate.

In [2]:
simObject = pc.ParticleTracing(flowFieldName='flows/F_Cell/DS2f005.DAT', NPAR=50)

Loading grids ... 
Interpolating ... Flow field loaded


There are various other, less important attributes in a **`ParticleTracing()`** object that we could specify.

- **`CROSS_MULT`**: (float) a multiplicative fine-tuning parameter which modifies the collision cross section of a helium atom w.r.t. one molecule of our species. The multiplier defaults to unity, and the reference value of the cross section is set at $\sigma_{ref}=20\cdot\sigma_{He-He}$, i.e. 20 times the helium-helium cross section.


- **`LITE_MODE`**: (bool) this flag defaults to `True`, whereby the simulation will not record the molecule's position in the output file UNTIL the molecule is sufficiently close to the cell aperture. In other words, if this flag is set to `False` the molecule position will be recorded at _every_ time-step, resulting in a much larger output file. 


- **`INIT_COND`**: (int) this parameter encodes the type of (pre-defined) initial conditions we would like to set for our molecules. The initial conditions might encode e.g. the region in the cell where the molecules are to start out, and the initial velocity distribution. These are hard-coded in the `ParticleTracing.initial_species_position()` and `ParticleTracing.initial_species_velocity()` methods. 


- **`CORRECTION`**: (int) this parameter specifies the kind of distribution we should use when generating inter-particle collision velocities. Currently defaults to `0`, indicating that an unbiased, "uncorrected" Maxwell-Boltzmann thermal distribution is used (this is the simplest/fastest distribution to draw from). If instead `CORRECTION=1` then the velocity distribution PDF has an "extra" factor of $v$ to account for the speed-dependence of the collision probability (see the reference document).  This dependence is hard-coded in `ParticleTracing.get_ambient_velocity()`.

For now we are using the default values for these "optional" parameters, and have only specified the flow field filename, `'flows/F_Cell/DS2f005.DAT'`, and the number of particles to simulate `NPAR=50` (this is just for example, we would typically want to simulate ~1000 molecules).

We have already declared our instance of the particle simulation class, `simObject`. We can now proceed to run a parallelized simulation for all (10) particles using the `ParticleTracing.parallel_main()` method. This function takes a single parameter `outfile`, which specifies an output file to write our results to.



In [3]:
outfile='Data/outputTest.dat'

simObject.parallel_main(outfile); #Runs parallelized simulations in simObject

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   22.9s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   24.0s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   24.0s
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:   25.1s remaining:   37.8s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   26.2s remaining:   26.2s
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   28.7s remaining:   19.1s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   31.8s remaining:   13.6s
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:   34.5s remaining:    8.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   37.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   38.1s finished


The output file will contain a list of 7-column rows. The seven columns correspond to three positions $x,y,z$, three velocities $v_x,v_y,v_z$, and a time $t$. Each row specifies a single particle, at a single time step. The file is organized into single-particle blocks, separated by rows of `0 0 0 0 0 0 0`. Each block starts and ends with the initial and final data for each particle, respectively.

To get a basic analysis of the particle data, we can use the **`fluidSim3D`** python program.

In [ ]:
import fluidSim3D as fs

Running the `analyzeTrajData()` method on our freshly made output file will give us basic statistics and plots about the final molecule distributions. However, the script relies on knowledge of the cell geometry (e.g. cell dimensions, aperture shape) to properly analyze the data, so we encode the geometry in the file name according to our usual format.

In [ ]:
!mkdir Data/FCellTesting
!cp Data/outputTest.dat Data/FCellTesting/f005.dat

Now we have our output file in the folder `FCellTesting`, with the filename `f005.dat` indicating that the simulation used cell geometry `f` with a flow rate of `5 SCCM`. 

In [ ]:
fs.analyzeTrajData(file_ext='f005.dat', folder='FCellTesting', plots=True)

If we now wanted to compare these results against simulations using a different cell geometry, e.g. hourglass-aperture geometry **`h`**, but with the same parameters, we can change the geometry of `simObject` using the method `ParticleTracing.set_flow_type()`, which takes as input the filename `ff` of the DSMC flow field we want to load.

In [ ]:
simObj.set_flow_type('flows/H_Cell/DS2h005.DAT')

This will set `simObj._geometry='h'`, and `simObj._flowrate=5`. With the new flow data loaded, we can again run a parallelized simulation of `NPAR=50` molecules.

In [ ]:
#Run simulation, store results in outfile2
outfile2 = 'Data/outputTest_H.dat'
simObj.parallel_main(outfile2); 

#Copy data to a new file which encodes geometry/flowrate in the filename
!mkdir Data/HCellTesting
!cp Data/outputTest_H.dat Data/HCellTesting/h005.dat


fs.analyzeTrajData(file_ext='h005.dat', folder='HCellTesting', plots=True) 